In [ ]:
# To use only Google Colab
# ! pip install matplotlib --upgrade

# Unidad II. Regresiones y reducción de dimensionalidad.

## Modelos de regresión lineales

Los modelos de regresión lineal permiten:

* Predecir una variable *dependiente*
* A partir de una combinación lineal de variables *independientes*

### Formulación general

$Y = \alpha + \beta_{1}X_{1} + \beta_{2}X_{2} + ... + \beta_{k}X_{k} + E$

Donde:

* $Y$: variable dependiente
* $X_{1}, X_{2}, ... , X_{k}$: variables independientes
* $\beta_{1}, \beta_{2}, ... , \beta_{k}$: coeficientes del modelo
* $\alpha$: intercepto u ordenada al origen
* $E$: error o residuo

### Objetivo del modelo

Encontrar los parámetros $\alpha, \beta_{1}, ..., \beta_{k}$ que mejor expliquen los datos.
Habitualmente se ajustan mediante **mínimos cuadrados**, minimizando:

$\sum_{i=1}^{N}(Y_{i} - \hat{Y}_{i})^{2} = \sum_{i=1}^{N} E_{i}^{2}$

### Supuestos del modelo

* **Linealidad**: la relación entre variables es una combinación lineal.
* **Independencia de los residuos**.
* **Homocedasticidad**: varianza constante del error.
* **Normalidad**: los residuos siguen una distribución normal con $\mu = 0$.
* **No colinealidad**: las variables independientes no deben ser combinaciones lineales entre sí.



### Residuos

Los residuos contienen toda la información sobre lo que la línea no capturó:

- ruido aleatorio
- relaciones no lineales
- variables omitidas
- errores de medición
- outliers

### Propiedades esperadas

- Si el modelo es adecuado, los residuos deberían:
- estar centrados alrededor de cero
- no mostrar patrones claros al graficarlos
- tener una variación similar (homocedasticidad)
- no estar correlacionados entre sí
- seguir aproximadamente una distribución normal

## Simulación de los datos de un modelo

Vamos a simular un modelo lineal completo.

$$ Y = \alpha + \beta_1 X_1 + \beta_2 X_2 + Errores$$

$$ Y = \alpha + B \times X + Errores$$

In [ ]:
# Vamos a seleccionar valores al azar para los parámetros del modelo
# luego vamos a simular las variables independientes y el error
# finalmente vamos a calcular los valores de la variable dependiente

import numpy as np
import scipy.stats as st

nvars = 2
nvalues = 1000

# Simular la intersección
alpha = np.random.rand()

# Simulación de los coeficientes
betas = np.random.rand(nvars)

# simular los errores
errors = 0.15 * st.norm.rvs(size=nvalues)

# Simular variables aleatorias
equis = np.random.rand(nvars, nvalues)

# Calcular la variable dependiente
Y = alpha + np.matmul(betas, equis)  + errors

In [ ]:
Y[:4]

### Inspección del modelo


In [ ]:
import matplotlib.pyplot as plt


fig, axes = plt.subplots(figsize=(12, 5))

sc = axes.scatter(equis[0], equis[1], c = Y)

axes.set_xlabel("Var independiente $X_0$")
axes.set_ylabel("Var independiente $X_1$")

cbar = fig.colorbar(sc, ax=axes)
cbar.set_label("Variable dependiente Y")

plt.show()

In [ ]:
import statsmodels.api as sm

exog = sm.add_constant(equis.T)

# Ordinal Least Square
reg_mod = sm.OLS(
    endog = Y,
    exog = exog
)

fitted = reg_mod.fit()

In [ ]:
# alpha, beta1, beta2
print(f"Fitted coefficients: {fitted.params}")
print(f"Real coefficients: {[alpha, betas]}")


In [ ]:
fitted.summary()

In [ ]:
# Como predecir nuevos valores?
#
# p.e, para X1 = 0.5 y X2 = 2

fitted.predict([1, 0.1, 2])

In [ ]:
var_with_intercept = sm.add_constant(equis.T)
print("Variables with estimated alpha:")
print(var_with_intercept[:4, :])

print(f"Predicted:\n{fitted.predict(var_with_intercept[:4, :])}")
print(f"Real values:\n{Y[:4]}")


In [ ]:
import matplotlib.pyplot as plt

plt.scatter(Y, fitted.predict(var_with_intercept))
plt.xlabel("Real data")
plt.ylabel("Predicted")
_ = plt.title(f"Coeficiente de determinación: $R^2 = {fitted.rsquared:0.3f}$")

In [ ]:
_ = plt.hist(fitted.resid, bins = 50)

In [ ]:
plt.scatter(
  fitted.predict(var_with_intercept),
  fitted.resid
)

### Coeficientes de determinación

Es el valor $R^2$ de la regresión. 
- Es la proporción de la varianza de la variables de respuesta que puede ser
  predicha a partir de las variables explicativas.
- Varía entre 0 y 1.
  - Cuanto mayor es, menos incerteza se tiene de los valores reales a partir del
   modelo.
- Suma de las diferencias de cuadrado de los datos:
  - $SS_{data} = \sum_i{(y_i-\bar{y})^2}$
- Suma de las diferencias de cuadrados de las predicciones:
  - $SS_{pred} = \sum_i{(fit(y_i)-\bar{y})^2}$
- $R^2 = \frac{SS_{pred}}{SS_{data}}$

Veamos como cambia el $R^2$ con el error del modelo.

In [ ]:
nvars = 2
nvalues = 50
variables = np.random.rand(nvars, nvalues)
exog = sm.add_constant(variables.T)
alpha = np.random.rand()
betas = np.random.rand(nvars)
fig, axes = plt.subplots(ncols=2, nrows=2)
axes = axes.flatten()
for i, err in enumerate([0.05, 0.2, 0.5, 1]):
  errors = st.norm.rvs(size=nvalues) * err
  y = alpha + betas@variables + errors
  reg_mod = sm.OLS(endog = y, exog = exog)
  fitted = reg_mod.fit()
  fval = fitted.fittedvalues
  axes[i].scatter(y,fval)
  axes[i].set_title(f"$R^2$: {fitted.rsquared:0.4f}")
fig.tight_layout()

### Mínimos Cuadrados Ponderados.

El **método de mínimos cuadrados ponderados (WLS, Weighted Least Squares)** es una extensión del método de **mínimos cuadrados ordinarios (OLS)** diseñada para corregir un problema típico de los modelos lineales: la **heterocedasticidad**.

En OLS, todos los puntos tienen **el mismo peso** al ajustar la recta o hiperplano.
Esto funciona bien si los residuos tienen **varianza constante** (homocedasticidad).

Pero si algunos puntos tienen **más ruido** que otros (heterocedasticidad):

* ciertos datos son mucho menos confiables,
* sin embargo, OLS les da el mismo peso que a puntos muy precisos,
* esto produce un modelo **ineficiente** y **sesga las inferencias**.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate synthetic data with heteroscedastic noise
np.random.seed(1)
x = np.linspace(0, 10, 50)
y_true = 2 * x + 3

# noise increases with x (heterocedastic)
noise = np.random.normal(0, 0.3 * x, size=x.shape)
y = y_true + noise

# Fit linear regression
coef = np.polyfit(x, y, 1)
y_pred = np.polyval(coef, x)

# Residuals
residuals = y - y_pred

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].scatter(x, y, s=15)
axes[0].plot(x, y_pred, c='red')
axes[0].set_xlabel("X")
axes[0].set_ylabel("Y")
for xi, yi, ypi in zip(x, y, y_pred):
  axes[0].plot([xi, xi], [yi, ypi], c ='green')
axes[0].plot([x[-3], x[-3]], [y[-3], y_pred[-3]], c ='salmon')

axes[0].annotate(
  xy = (9.6, 25),
  text = "Residuo",
  xytext = (7, 26),
  arrowprops = {
    "arrowstyle": "->"
  }
)

axes[1].scatter(x, residuals, s=15)
axes[1].plot([x[-3], x[-3]], [0, residuals[-3]], c ='salmon')
axes[1].axhline(0)
axes[1].set_xlabel("X")
axes[1].set_ylabel("Residuos")


plt.suptitle("Modelo lineal con heterocedasticidad de los residuos")
plt.show()



WLS dice:

- No todos los puntos valen lo mismo.
- Los puntos más ruidosos deben influir **menos**.
- Los puntos más precisos deben influir **más**.

Así, cada observación recibe un **peso wᵢ**.

* Si un dato tiene mucha varianza (es poco confiable), se le da **poco peso**.
* Si un dato tiene poca varianza (es muy confiable), se le da **mucho peso**.

En lugar de minimizar

$ \sum_{i}(Y_i - \hat{Y}_i)^2 $

WLS minimiza:

$ \sum_{i=1}^{N} w_i (Y_i - \hat{Y}_i)^2 $

donde $w_i$ es el peso asignado a cada observación.

En la práctica:

* Si se conoce la varianza del error de cada punto:

  $ w_i = \frac{1}{\text{Var}(E_i)} $

  Es decir, **menos varianza = más peso**.

* Si no (casi siempre), se puede **estimar la forma de la heterocedasticidad** a partir de los residuos de un modelo OLS preliminar.

  Esto se llama **WLS iterativo (IWLS)**.

Ejemplo típico:

Si se observa que

$ Var(E_i) \propto X_i^2 $

entonces se usan pesos

$ w_i = \frac{1}{X_i^2} $


### Ejercicio

Estudio del crecimiento de plantas (en cm) en función de:

- Sunlight_hours: horas de luz solar al día
- Water_ml: cantidad de agua diaria en mililitros
- Nitrogen_mgL: concentración de nitrógeno en la solución nutritiva (mg/L)
- Growth_cm: crecimiento del tallo (variable dependiente Y)

| id | Sunlight_hours |	Water_ml | Nitrogen_mgL |	Growth_cm |
| -  | --- | ---- | --- | --- |
| 1 | 5.62 | 414 | 24.39 | 13.64 |
| 2 | 9.66 | 180 | 20.50 | 13.73 |
| 3 | 8.12 | 306 | 23.79 | 12.81 |
| 4 | 7.19 | 337 | 22.90 | 11.62 |
| 5 | 4.09 | 119 | 16.96 | 4.74 |
| 6 | 4.09 | 343 | 23.44 | 11.87 |
| 7 | 3.41 | 168 | 6.77 | 5.16 |
| 8 | 9.06 | 126 | 8.92 | 9.23 |
| 9 | 7.21 | 480 | 5.90 | 13.22 |
| 10 | 7.96 | 486 | 11.51 | 11.61 |
| 11 | 3.14 | 423 | 12.77 | 9.92 |
| 12 | 9.79 | 222 | 10.43 | 12.06 |
| 13 | 8.83 | 139 | 21.57 | 10.13 |
| 14 | 4.49 | 374 | 12.14 | 9.03 |
| 15 | 4.27 | 276 | 10.62 | 9.23 |
| 16 | 4.28 | 149 | 15.85 | 8.49 |
| 17 | 5.13 | 298 | 7.82 | 10.50 |
| 18 | 6.67 | 114 | 21.04 | 10.14 |
| 19 | 6.02 | 464 | 6.49 | 15.49 |
| 20 | 5.04 | 204 | 24.74 | 8.40 |
| 21 | 7.28 | 365 | 20.44 | 11.95 |
| 22 | 3.98 | 225 | 8.97 | 6.68 |
| 23 | 5.05 | 308 | 5.11 | 5.75 |
| 24 | 5.56 | 319 | 21.31 | 12.40 |
| 25 | 6.19 | 174 | 19.14 | 10.92 |

## Ejercicio

Se tienen mediciones de expresión del genY en un cultivo celular y
se quiere generar un modelo lineal que permita explicar su expresión
En base a la medición de la expresión de otros tres genes (genA, genB, y genC).

- Genera un modelo lineal simple (OLS)
- Revisa la distribucón de los residuos
- Revisa el comportamiento de los residuos con cada variable aleatoria

- ¿Hay evidencia de que la homocedasticidad no se cumple?

- En caso de que no se cumple, usar un modelo ponderado


In [ ]:
geny = [3.086837800510745, 2.4146832358765127, 0.8326379590980215, -0.18756563957318906, -1.0129559117683014, 0.08472057038503067, 2.6130082514548736, -0.8176505845473698, 0.5435663010685376, 0.3817455704467031, 1.583757677011703, 0.6000696809356555, 0.4743183046234437, 0.48986626995497773, 1.0507474798816785, 2.833217285053774, 0.12657831414800047, 0.27318705622685846, 1.438566075336698, -0.08180378486451778, 1.1962885656238522, -0.15914795598275155, -0.7537991616766773, 1.7157611093825857, -0.14224551799549467, 1.7071992707314803, 0.36286860122069964, 0.016201016136538238, 1.6187633248855526, 1.0765880346431511, -0.22302339214654432, 1.561169609354351, 0.24706422119394555, 0.9316644843332742, 1.0585838147102284, 0.16660494086543787, 0.5692873342109648, 1.7278803116955932, 1.4134448428695903, 0.6899814572943785, 1.510898102920649, 1.475815464902645, 1.0358477031447522, 2.181589099925082, 1.3481673708636162, 1.017409464134297, 0.9200085026486999, 1.6573074002005987, 1.8236819360642467, 0.8718552365026836, 0.21843089243978997, 1.2341481030812687, 0.6990712519206584, 0.36016187507986336, 0.15601957838193325, 1.624202672023532, 0.5604390376766902, 1.2077858735878029, 0.6915832218939038, 1.7155200057736444, 0.6289824796551078, 1.2608880482942357, 1.9035596814140356, 0.7789185305484503, 2.0905713973104523, 0.4869262576786929, 2.2492834784712707, 0.8103515132678928, 0.5827996975732936, 0.32646179532556496, 0.8296468679642989, 0.5693018192634287, 2.2447664506613174, 0.5368659833270509, 1.5935262460174253, -0.012768688474380285, 1.0936597619665505, 2.1909156339228217, -0.0437106361780184, 2.9234007913728512, -0.8111319546321792, -0.07642225988281814, 0.678835581844105, 1.4660085667037186, 1.6779173332052513, 0.39782162805963917, 0.7950634854645009, 1.0683506094434523, 0.2158699339388257, -0.05744586598143475, 2.9247846922496876, 1.936866466289612, 0.20887418881351438, 1.184255312888161, 3.0514381872601968, 3.1878996016537475, 3.2314865913350896, 0.10626093314422413, 2.7190713279673986, 0.3091251717365089]
genA = [0.03637285576135929, 0.05927181360230671, 0.061556920524046044, 0.07427721027712897, 0.08052605191527085, 0.08560174775721296, 0.08757250279022832, 0.09230949302900593, 0.097288819218175, 0.10407226495895416, 0.11443520825997311, 0.1220926555559132, 0.1263627987090422, 0.13566730992552467, 0.14137202923341952, 0.14145040312580726, 0.14791502288576253, 0.15466925480608273, 0.16182138330932827, 0.1650813855658283, 0.1919269345993979, 0.19479132935636734, 0.21172775936732025, 0.22172524215579936, 0.23830609913892387, 0.24349603462013047, 0.245302701563329, 0.24607555492416577, 0.2585775161010281, 0.2631188950041533, 0.26508439586284316, 0.2951257599493058, 0.2963867161554098, 0.29660315990906505, 0.32150704593404156, 0.35517633066673193, 0.3560857388142352, 0.35804634873818275, 0.3705091175190778, 0.39856517669057057, 0.4022926302868761, 0.4179321711047578, 0.4226292492874031, 0.42347033574322557, 0.43093542602831203, 0.4323918013092254, 0.4340859454203426, 0.4402285655559619, 0.4556384736742697, 0.46497296743521777, 0.4785309354258952, 0.4831009682483196, 0.49847790216743415, 0.5150943981093575, 0.5212437357075573, 0.552641909472555, 0.5543887647914784, 0.5594848925038969, 0.6196557334225025, 0.6403768174962349, 0.6412499052761262, 0.6434105782172138, 0.6487969569870742, 0.649691761759749, 0.6629064051571036, 0.6641554999963029, 0.6721814984306793, 0.6728626399769783, 0.693445857984454, 0.6948199294003111, 0.7234803197496669, 0.7242832743740774, 0.7446797200984736, 0.7447313938274647, 0.7606779076263023, 0.7679547202426771, 0.8018446851177204, 0.8094367722230813, 0.8132708472427249, 0.8321210646838488, 0.8378145337391567, 0.843726417356169, 0.854366137103977, 0.8632215897915169, 0.8687368012922272, 0.8713237452019199, 0.908054630894359, 0.9261989718610584, 0.9492007228915365, 0.9509101278447166, 0.95452531445416, 0.9565886344053418, 0.9597978453498803, 0.9598439823786653, 0.960080901134645, 0.9707041420997781, 0.9747881788815097, 0.9795324812150754, 0.9906911405500243, 0.9999380798496119]
genB = [0.5926266015453873, 0.6477266458845181, 0.7357761066681715, 0.817591163029072, 0.9991085625937363, 0.2755796118866999, 0.6661550249501927, 0.040814532429251704, 0.9596523220421151, 0.14932356240608746, 0.009637981506005922, 0.9763978174854533, 0.7611649920476607, 0.07517810831723704, 0.425045781653004, 0.06325908816918702, 0.06841643489355609, 0.58876310531655, 0.06242399741680282, 0.3647241346125316, 0.5133463931836966, 0.3460767937229867, 0.1385958196308965, 0.5381047749462966, 0.9754288599974401, 0.1078100910127393, 0.6146055221572481, 0.031902984707201765, 0.45250589177372436, 0.6284814660983264, 0.5194286916901846, 0.5362389354963608, 0.08792208833707671, 0.942952605128808, 0.014454790962893704, 0.23277605499605414, 0.8012015524089412, 0.2653408474004182, 0.33533227964521173, 0.3387807883354016, 0.6291881786646139, 0.5717417196177603, 0.6324712316631381, 0.9250011155426164, 0.5217236315918775, 0.8410664495365993, 0.056041287949205376, 0.7239559768217587, 0.9670358088701315, 0.31737965529835965, 0.2901130978373946, 0.7655986881198623, 0.5735886637674851, 0.389793444716802, 0.019920608072904855, 0.2818548366501852, 0.364749364164615, 0.22119717294697827, 0.793620578035634, 0.013822474652193995, 0.6210693772367932, 0.5138910917588625, 0.6271080294671803, 0.5297009289083621, 0.9594686017085301, 0.1597257093354736, 0.7212580548853585, 0.010750326605821536, 0.2650082131251912, 0.35314697088438995, 0.16885129493690276, 0.6634470050933876, 0.7642426435078186, 0.5520423169074016, 0.49765443595954206, 0.41627682740733385, 0.6970657346834495, 0.614193505725828, 0.019657634779003952, 0.2972918904952152, 0.5240191399239368, 0.35212253275786454, 0.535219077256922, 0.9594893777199524, 0.6509569334855294, 0.01913546275246203, 0.9559630689454027, 0.9271867867972314, 0.6669537620066552, 0.9729757680381915, 0.38773682336415294, 0.5962273225937479, 0.9554856175005578, 0.7268038890961867, 0.18051317249252374, 0.7383199778282702, 0.7234588923010279, 0.9760247381851874, 0.24492394112285487, 0.6772495039496755]
genC = [0.8580112785515976, 0.3868449009637507, 0.593986653012601, 0.9714279325148181, 0.09838720018773495, 0.06419759792003077, 0.6501277611678716, 0.05573490447737928, 0.4373392956742551, 0.485916536427105, 0.395552567766702, 0.8299384037992457, 0.4782854262192062, 0.2524508604543523, 0.7687281433873353, 0.8942162255499718, 0.957576310776935, 0.6819778804315266, 0.3736935930655215, 0.05630451749281806, 0.7688794094816341, 0.745878880946871, 0.15884113657456078, 0.13637604205948695, 0.1971017697797608, 0.8812698520957518, 0.013756634591799988, 0.34096397954926294, 0.24267901507927658, 0.005550194543500919, 0.33271233447211035, 0.6173747739044531, 0.40608647632404715, 0.938154348184007, 0.9565573295774848, 0.7789243897827737, 0.24940834561834402, 0.389638772798343, 0.6988838081555323, 0.8327354004049794, 0.9899530654767035, 0.7609533002202779, 0.9733600680704982, 0.7642801077111387, 0.8808008025620017, 0.6728773343739323, 0.2870010391929436, 0.6681070966737158, 0.6035738423588519, 0.9260458946781281, 0.2336545383166505, 0.35421588262755976, 0.34292965344336157, 0.23276719164053683, 0.4133427592664234, 0.7842075313106078, 0.03429110851655082, 0.375446699166437, 0.7171345313661784, 0.6873852718160514, 0.30233140425065486, 0.20958784447178358, 0.9371745923381894, 0.9031784632114025, 0.9758355754924247, 0.9066872087976177, 0.5533890150441751, 0.07119988868032257, 0.6660289598284479, 0.7113930780295445, 0.8065632798254271, 0.9961648834425902, 0.5401239340043108, 0.3591831179701859, 0.5758173337687532, 0.890907271188484, 0.8631735114895481, 0.06635020049197904, 0.8831993836072132, 0.847198008862226, 0.08478045445481797, 0.5436942669446431, 0.958959247121608, 0.5278847606091703, 0.5822100413783694, 0.7952897548498099, 0.611520644384133, 0.27645987207252487, 0.25912707669939394, 0.14051817758334895, 0.3135850102926674, 0.8174768223743901, 0.9216110121769271, 0.803110822787875, 0.3213623704518963, 0.3932841049150033, 0.5874809907180399, 0.2843808607865316, 0.08907526593100401, 0.6836854160511551]


## Métodos de Regularización: Ridge, Lasso y Elastic Net

Los modelos lineales pueden sobreajustarse cuando hay muchas variables o alta correlación entre ellas.
Los métodos de regularización agregan una penalización sobre los coeficientes para controlar la complejidad del modelo.

### Ridge Regression (L2)

* Penaliza la suma de los cuadrados de los coeficientes.
* Tiende a **reducir los coeficientes**, pero no los lleva exactamente a cero.
* Útil cuando hay **multicolinealidad**.
* Función de costo:
  $J(\beta)=\sum(Y-\hat{Y})^2+\lambda\sum\beta_j^2$

### Lasso Regression (L1)

* Penaliza la suma del valor absoluto de los coeficientes.
* Puede llevar coeficientes exactamente a **cero**, realizando **selección de variables**.
* Adecuado cuando solo algunas variables son realmente importantes.
* Función de costo:
  $J(\beta)=\sum(Y-\hat{Y})^2+\lambda\sum|\beta_j|$

### Elastic Net (L1 + L2)

* Combina las penalizaciones de Ridge y Lasso.
* Permite seleccionar variables como Lasso, pero es más estable cuando las variables están correlacionadas.
* Controlado por dos parámetros:

  * $\lambda$: intensidad total de la regularización
  * $\alpha$: proporción L1 vs L2
* Función de costo:
  $J(\beta)=\sum(Y-\hat{Y})^2+\lambda\left[\alpha\sum|\beta_j|+(1-\alpha)\sum\beta_j^2\right]$



## Buscando colinearidad


In [ ]:
import sklearn.datasets as datasets

iris = datasets.load_iris(as_frame=True)

iris_m = iris["frame"].iloc[:, :4].copy()
iris_m

# Queremos estimar sepal_length a patir de las otras variables.

iris_m.corr()

In [ ]:
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt


# Defino los variables dependientes e independientes.
x = iris_m[["sepal width (cm)", "petal length (cm)", "petal width (cm)"]]
y = iris_m["sepal length (cm)"]

# Defino el modelo
model = Ridge(alpha = 0, fit_intercept=True)
model2 = Ridge(alpha = 5, fit_intercept=True)

# Hago el ajuste del modelo
fitted = model.fit(x, y)
fitted2 = model2.fit(x, y)

fig, axes = plt.subplots()

axes.set_xlabel("Valores reales")
axes.set_ylabel("Valores predichos")
_ = axes.scatter(y, fitted.predict(x), marker=".")
_ = axes.scatter(y, fitted2.predict(x),marker=".")


In [ ]:
print(fitted.coef_, fitted.intercept_, model.score(x, y))
print(fitted2.coef_, fitted2.intercept_, model2.score(x, y))

## Ejercicio

Comparar el ajuste de los datos a modelos Ridge variando el parámeto $\lambda$.

Ajustar los datos del dataset iris, para predecir "sepal length" en base a las
otras variables numéricas. Hacer un barrido de los valores $\lambda$ desde 0 a
1000 (al menos 10 puntos intermedios).

Compara los valores $R^2$, y los coeficientes del modelo obtenidos.
